In [25]:
# This script is used for interacting with OpenLCA.
# Codes are adapted from Julian Rickert @GreenDelta by Ao Chen @ETH.
# Connect to IPC server in OpenLCA software before running. (Port: 8080)
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})


import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [26]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list)),
                               columns=['name', 'id'])
processes_df

,name,id
0,"heat production, heavy fuel oil, at industrial...",4f582f90-d0ae-3552-be43-70c730e82af0
1,"containerboard production, fluting medium, rec...",fdffa34a-17fa-3676-9941-0a8a2bc5f5ac
2,"market for wood chips, dry, measured as dry ma...",35ca5293-f067-326c-ae45-920013c734c7
3,"treatment of waste glass, unsanitary landfill,...",ab082ee7-7469-3a1f-8e8f-f71328b9e842
4,"heat and power co-generation, natural gas, con...",3c7c0ba3-acd4-3e19-be88-8fff9dc87788
...,...,...
19123,"steel production, electric, chromium steel 18/...",39b4f8c5-a396-3e00-b8c1-db951800e388
19124,"market for glued laminated timber, PUR-glue | ...",c8a2e84f-6e92-3caa-ab1f-388427902eec
19125,"electricity production, hard coal | electricit...",ea622ed5-24a9-3c50-adbb-6f8eb7465bff
19126,"land already in use, annual cropland to forest...",8a8f8368-b831-3525-a257-065cacf08853


In [27]:
# Search specific process databases with key word(s)
# Note that the search is blankspace-sensitive
search_process_df = processes_df[processes_df['name'].str.contains(r'high voltage, production mix(?!$)')] # TODO: change keywords
search_process_df.reset_index(drop=True, inplace=True)

search_process_df

,name,id
0,"electricity, high voltage, production mix | el...",9e9e785d-fa84-34a0-8e2d-c43d37166a8e
1,"electricity, high voltage, production mix | el...",8fd2d8ca-2eb5-361b-8e29-ec4aafcd081d
2,"electricity, high voltage, production mix | el...",becdeb83-db53-3fca-8818-1ed18cacdba6
3,"electricity, high voltage, production mix | el...",6d09283d-34c4-3939-85b7-849819c941d0
4,"electricity, high voltage, production mix | el...",8ec2d93a-b7d4-35d1-9059-a29e4f93b720
...,...,...
109,"electricity, high voltage, production mix | el...",dc1c6a45-324e-3e60-8123-ef2aaf7e6001
110,"electricity, high voltage, production mix | el...",1d3a4b99-50d6-3c30-9ece-7c10d2c3d938
111,"electricity, high voltage, production mix | el...",3380a306-abf9-32c1-817d-6ddf33ba49e7
112,"electricity, high voltage, production mix | el...",fad3093c-f1df-313f-a0ae-4386d9c837b3


In [6]:
# Get all flow dataset in database
flow_descriptor = client.get_descriptors(olca.Flow)
flow_list = []
id_list = []

for flow in flow_descriptor:
    flow_list.append(flow.name)
    id_list.append(flow.id)
flows_df = pd.DataFrame(list(zip(flow_list,
                                   id_list)),
                               columns=['name', 'id'])
# flows_df

In [7]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_flow_df = flows_df[flows_df['name'].str.contains(r'oil(?!$)')] # TODO: change keywords
search_flow_df.reset_index(drop=True, inplace=True)

# search_flow_df

In [28]:
# Create product systems for selected processes
search_list_processID = list(search_process_df['id'])
product_system_ids = []
product_system_names = []
for i in range(len(search_list_processID)):
    product_system = client.create_product_system(search_list_processID[i], default_providers='prefer', preferred_type='SYSTEM_PROCESS')
    product_system_ids.append(product_system.id)
    ps = client.find(olca.ProductSystem, search_process_df['name'][i])
    product_system_names.append(ps.name)

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids)), 
                    columns=['product system name', 'product system id'])
product_system_df   

,product system name,product system id
0,"electricity, high voltage, production mix | el...",c549b3f7-a144-4431-bcc4-df11857ffeb4
1,"electricity, high voltage, production mix | el...",3aa28ecb-1b57-481c-86d1-142462ad619f
2,"electricity, high voltage, production mix | el...",98b9b4db-f0bb-4288-8c04-0f91925b35b3
3,"electricity, high voltage, production mix | el...",0ba71b8a-6c24-4224-bac7-6bde03999bd1
4,"electricity, high voltage, production mix | el...",6987a08b-057e-405c-81d1-343a4fa06645
...,...,...
109,"electricity, high voltage, production mix | el...",268fe246-b8f9-47ff-958d-8c3fbc9553ef
110,"electricity, high voltage, production mix | el...",d2205364-6db4-4f03-aa5f-242b5d708aa0
111,"electricity, high voltage, production mix | el...",dd9d80b5-f20e-4288-9cdb-91ec2116d67f
112,"electricity, high voltage, production mix | el...",3a7c3cc5-748d-42ad-ac1b-367129115144


In [30]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.CONTRIBUTION_ANALYSIS
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013')

# amount is the amount of the functional unit (fu) of the system that
# should be used in the calculation; unit, flow property, etc. of the fu
# can be also defined; by default openLCA will take the settings of the
# reference flow of the product system
setup.amount = 1.0

In [31]:
setup

CalculationSetup(id='', olca_type='CalculationSetup', calculation_type=<CalculationType.CONTRIBUTION_ANALYSIS: 'CONTRIBUTION_ANALYSIS'>, product_system=None, impact_method=Ref(id='11e3b622-f745-4742-ad90-57c3efeef416', olca_type='ImpactMethod', name='IPCC 2013', description=None, version=None, last_change=None, category_path=None, library=None, ref_unit=None, location=None, flow_type=None, process_type=None), with_costs=None, with_regionalization=None, nw_set=None, allocation_method=None, parameter_redefs=None, amount=1.0, unit=None, flow_property=None)

In [33]:
for ps in range(len(product_system_names)):
    setup.product_system = client.find(olca.ProductSystem, product_system_names[ps])

    calc_result = client.calculate(setup)

    client.excel_export(calc_result, 'calc_result.xlsx')

    print('Remaining number of processes to be calculated: '+str(ps))

In [2]:
import pandas as pd

filename_xlsx = 'calc_result.xlsx'
result_db = pd.ExcelFile(filename_xlsx)
impact_contributions_df = result_db.parse('Process impact contributions')

/Users/ocean/opt/miniconda3/envs/suscons/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
impact_contributions_df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13514,Unnamed: 13515,Unnamed: 13516,Unnamed: 13517,Unnamed: 13518,Unnamed: 13519,Unnamed: 13520,Unnamed: 13521,Unnamed: 13522,Unnamed: 13523
0,NaN,NaN,NaN,NaN,Process UUID,42e2dfeb-b105-36b2-bdb9-64b88243125d,f41ef2f1-fcd2-3caf-bce4-704e05af6574,fcf136e8-ec34-3ea3-9e9c-4c071308ffc6,ac81257d-f5b3-31f0-86e6-0f8f8c64cc71,7f7a7e23-c352-31d4-8b44-16c73d88df87,...,89592830-64e1-367e-8d17-be69ab4afdc0,69cb17d9-7a02-395a-b694-f1b0578384a1,19967c42-3567-3f73-8431-819601aedc15,e4683e5a-fd60-35cc-bdb4-db6b78d99e86,b4642218-e747-3a51-bc11-566edbb167ba,7ab83df0-45d1-3549-981a-edd8d2571f92,81589038-a6db-398c-8d9a-5f81d0e27ae2,aaf12077-a4fc-3aaa-a20e-de47ddfceb54,82832cc3-e392-32d1-b5f8-eab71964997d,6018495b-0854-35ef-9197-716c1920731e
1,NaN,NaN,NaN,NaN,Process,"electricity, high voltage, production mix | el...","1,1-difluoroethane production, HFC-152a | 1,1-...","1,1-difluoroethane production, HFC-152a | 1,1-...","1,1-dimethylcyclopentane to generic market for...","1-propanol production | 1-propanol | Cutoff, U",...,zinc monosulfate production | zinc monosulfate...,"zinc oxide production | zinc oxide | Cutoff, U","zinc oxide production | zinc oxide | Cutoff, U","zinc scrap, post-consumer, Recycled Content cu...",zinc sulfide production | sulfuric acid | Cuto...,zinc sulfide production | sulfuric acid | Cuto...,zinc sulfide production | zinc sulfide | Cutof...,zinc sulfide production | zinc sulfide | Cutof...,zirconium oxide production | zirconium oxide |...,zirconium oxide production | zirconium oxide |...
2,NaN,NaN,NaN,NaN,Location,US-MRO,RoW,US,GLO,RoW,...,RER,RoW,RER,GLO,RER,RoW,RER,RoW,RoW,AU
3,NaN,Impact category UUID,Impact category,Reference unit,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,dea0debe-1f03-3356-89cd-813a90145c75,climate change - GTP 100a,kg CO2-Eq,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,b31d844a-5290-39f4-89d6-c3762b20dbea,climate change - GTP 20a,kg CO2-Eq,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,f5a05383-61ed-3665-a747-cf966669ab6a,climate change - GWP 100a,kg CO2-Eq,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,36f33293-0034-3939-a2e0-66b370a0a7a8,climate change - GWP 20a,kg CO2-Eq,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
